# 为什么使用jupyter notebook

Jupyter Notebook 特别适合数据科学和机器学习领域的开发，因为它具有以下优势：

交互性：Jupyter Notebook 可以让用户在每个代码块中进行交互式编程，这使得调试和调整代码变得更加容易和直观。数据科学和机器学习的开发过程通常需要进行大量的试错和实验，使用 Jupyter Notebook 可以快速迭代和调整代码。

可视化：Jupyter Notebook 可以轻松地生成丰富的数据可视化图表，这对于数据科学和机器学习领域的分析和探索非常重要。用户可以使用 Python 库（如 Matplotlib、Plotly 和 Bokeh）来创建交互式图表和可视化效果。

文档化：Jupyter Notebook 可以将代码、文本和图像组合在一起，形成一份完整的文档，这对于数据科学和机器学习领域的研究和报告非常有用。用户可以使用 Markdown 语法来编写格式化的文本，并在其中嵌入代码和图像。

社区支持：Jupyter Notebook 的社区非常活跃，有大量的第三方扩展库和插件可供使用。这些扩展库可以提供额外的功能，如支持不同的编程语言、增强代码编辑和调试、提供更丰富的数据可视化等。

可分享性：Jupyter Notebook 可以轻松地分享给其他人，这使得团队协作和知识共享变得更加容易。用户可以将 Jupyter Notebook 导出为 HTML、PDF 或 Markdown 格式，或者上传到在线服务（如 GitHub、GitLab 和 Jupyter Notebook Viewer）进行分享和协作。

# 参数管理

In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
x = torch.rand(size=(2, 4))
print(net(x))

tensor([[-0.6147],
        [-0.7002]], grad_fn=<AddmmBackward0>)


## 参数访问

In [5]:
print(net[2].state_dict())  
# 拿到的是最后的一个输出层nn.Linear(8, 1)
# state_dict是一个状态，权重可以被改变

OrderedDict([('weight', tensor([[-0.2827, -0.0224, -0.0247,  0.1026, -0.3062,  0.0252,  0.0782, -0.0797]])), ('bias', tensor([-0.3479]))])


## 目标参数

In [6]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)  # 值本身，还有一个梯度

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3479], requires_grad=True)
tensor([-0.3479])


In [7]:
net[2].weight.grad == None

True

## 一次性访问所有参数

In [8]:
# 拿出第一层的named_parameters的参数
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

# 把整个网络的parameters全部拿出来
print(*[(name, param.shape) for name, param in net.named_parameters()])
# 因为1是ReLu，它是没有参数的

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [9]:
net.state_dict()['2.bias'].data  # 通过名字获取参数

tensor([-0.3479])

## 从嵌套块收集参数

In [11]:
def block1():
    return nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f"block {i}", block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(x)

tensor([[0.0982],
        [0.0982]], grad_fn=<AddmmBackward0>)

最后rgnet(x)输出为2*1，是因为x是行数为2的矩阵，而1是因为定义的最后linear层的输出特征为1

In [12]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)
